In [107]:
import os
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from functools import partial
from sklearn.metrics import mean_squared_error, f1_score

In [110]:
DATA_DIR = "/home/gangda/workspace/ds-richter/data"

catboost_version = 2
lightgbm_version = 1
num_fold = 4

y = pd.read_csv(DATA_DIR + '/train_labels.csv', index_col='building_id')
y_catboost = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(num_fold, catboost_version))
y_lightgbm = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(num_fold, lightgbm_version))

y_catboost_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(num_fold, catboost_version))
y_lightgbm_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(num_fold, lightgbm_version))

In [209]:
DATA_DIR = "/home/gangda/workspace/ds-richter/data"


y_catboost_1 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(3, 1))
y_catboost_2 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(4, 2))
y_catboost_3 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(5, 1))
y_catboost_3_2 = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_train.npy'.format(5, 2))
y_lightgbm_1 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(3, 1))
y_lightgbm_2 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(4, 1))
y_lightgbm_3 = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_train.npy'.format(5, 1))

y_catboost_1_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(3, 1))
y_catboost_2_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(4, 2))
y_catboost_3_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(5, 1))
y_catboost_3_2_test = np.load(DATA_DIR + '/intermediate/catboost_{}fold_v{}_test.npy'.format(5, 2))
y_lightgbm_1_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(3, 1))
y_lightgbm_2_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(4, 1))
y_lightgbm_3_test = np.load(DATA_DIR + '/intermediate/lightgbm_{}fold_v{}_test.npy'.format(5, 1))

In [213]:
y_prob = np.stack([y_catboost_1, y_catboost_2, y_catboost_3, y_catboost_3_2, y_lightgbm_1, y_lightgbm_2, y_lightgbm_3])

y_prob = np.log1p(y_prob)
y_prob = np.sum(y_prob, axis=0)
y_pred = y_prob.argmax(axis=1) + 1
f1_score(y, y_pred, average='micro')

0.7538344058541602

In [129]:
# sns.distplot(y_pred)

In [50]:
from sklearn.metrics import classification_report, confusion_matrix

# mixure
# print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))

[[ 14106  10658    360]
 [  5581 125772  16906]
 [   601  30064  56553]]


In [144]:
# catboost
# print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))

[[ 13758  11036    330]
 [  5393 126094  16772]
 [   564  30526  56128]]


In [147]:
# lightgbm
# print(classification_report(y, y_pred))
print(confusion_matrix(y, y_pred))

[[ 14299  10351    474]
 [  6075 124571  17613]
 [   680  29535  57003]]


In [180]:
class GeneralizedMeanBlender:
    """Combines multiple predictions using generalized mean"""
    def __init__(self, p_range=(-2,2)):
        """"""
        self.p_range = p_range
        self.p = None
        self.weights = None

    def _objective(self, trial, X, y):

        # create hyperparameters
        p = trial.suggest_float(f"p", *self.p_range)
        weights = [
            trial.suggest_float(f"w{i}", 0, 1)
            for i in range(X.shape[0] * 1)
        ]
        weights = np.reshape(weights, (X.shape[0], 1, 1))

        # blend predictions
        if p <= 0:
            blend_preds = np.log1p(X) * weights
        else:
            blend_preds = X**p * weights
        blend_preds = np.sum(blend_preds, axis=0)

        y_pred = blend_preds.argmax(axis=1) + 1
        return f1_score(y, y_pred, average='micro')

    def fit(self, X, y, n_trials=10):
        # optimize objective
        obj = partial(self._objective, X=X, y=y)
        study = optuna.create_study(direction="maximize")
        study.optimize(obj, n_trials=n_trials)
        # extract best weights
        if self.p is None:
            self.p = [v for k,v in study.best_params.items() if "p" in k][0]
        self.weights = np.array([v for k,v in study.best_params.items() if "w" in k])
        self.weights /= self.weights.sum()

        self.study = study

    def transform(self, X):
        assert self.weights is not None and self.p is not None, \
            "Must call fit method before transform"
        weights = np.reshape(self.weights, (X.shape[0], 1, 1))
        if self.p <= 0:
            prob = np.log1p(X) * weights
        else:
            prob = X**self.p * weights
        prob = np.sum(prob, axis=0)
        return prob.argmax(axis=1) + 1

    def fit_transform(self, X, y, **kwargs):
        self.fit(X, y, **kwargs)
        return self.transform(X)

In [221]:
# X_prob = np.stack([y_catboost, y_lightgbm])
X_prob = np.stack([
    y_catboost_1, y_catboost_2, y_catboost_3, y_catboost_3_2,
    y_lightgbm_1, y_lightgbm_2, y_lightgbm_3
])
y_prob_test = np.stack([
    y_catboost_1_test, y_catboost_2_test, y_catboost_3_test, y_catboost_3_2_test,
    y_lightgbm_1_test, y_lightgbm_2_test, y_lightgbm_3_test
])

gmb = GeneralizedMeanBlender()
gmb.fit(X_prob, y, n_trials=10000)

[I 2023-04-10 17:25:35,952] A new study created in memory with name: no-name-61f2a448-6b7b-4447-acc1-2c7a82cc39a8
[I 2023-04-10 17:25:36,017] Trial 0 finished with value: 0.7539610362201219 and parameters: {'p': -0.5123213726409293, 'w0': 0.3477186962622182, 'w1': 0.23212173462841657}. Best is trial 0 with value: 0.7539610362201219.
[I 2023-04-10 17:25:36,091] Trial 1 finished with value: 0.7535197485811643 and parameters: {'p': 0.0909662881356832, 'w0': 0.691525067151418, 'w1': 0.2408327154112031}. Best is trial 0 with value: 0.7539610362201219.
[I 2023-04-10 17:25:36,156] Trial 2 finished with value: 0.7519809977705381 and parameters: {'p': -1.2171645372943467, 'w0': 0.024841296088730647, 'w1': 0.6608443678878638}. Best is trial 0 with value: 0.7539610362201219.
[I 2023-04-10 17:25:36,220] Trial 3 finished with value: 0.7538727786923304 and parameters: {'p': -1.6928273258348407, 'w0': 0.9800233009985341, 'w1': 0.45780154100259995}. Best is trial 0 with value: 0.7539610362201219.
[I 2

In [226]:
f1_score(y, gmb.transform(X_prob), average='micro')

0.7541567376947901

In [225]:
print(gmb.weights, gmb.p)

[0.56376082 0.43623918] 1.9006709774195878


In [194]:
fig = optuna.visualization.plot_param_importances(gmb.study)
fig.show()

## Submit

In [220]:
test_preds = gmb.transform(y_prob_test)

submission_format = pd.read_csv(DATA_DIR + '/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=test_preds,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission['damage_grade'] = my_submission['damage_grade'].astype(int)
my_submission.to_csv(DATA_DIR + '/submission/mixture_{}.csv'.format(7550))